In [3]:
import numpy as np
import pandas as pd
from one.api import ONE

from psyfun import io

/home/crombie/miniconda3/envs/iblenv/lib/python3.10/site-packages/one/alf/files.py:10: FutureWarning: `one.alf.files` will be removed in version 3.0. Use `one.alf.path` instead.
  warnings.warn(


In [2]:
mode = 'auto'  # sets mode for all queries (one.seach* methods)
one = ONE(mode=mode)

Downloading: /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/tmphgjeo6np/cache.zip Bytes: 139632859


100%|█████████████| 133.1642713546753/133.1642713546753 [00:25<00:00,  5.25it/s]


In [14]:
# Load recordings from previous query
df_recordings = io.load_recordings()
eids = df_recordings['eid'].unique()
eid = eids[0]

In [28]:
# List video-related datasets
[dataset for dataset in one.list_datasets(eid) if 'Camera' in dataset]

['alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/rightCamera.ROIMotionEnergy.npy',
 'raw_video_data/_iblrig_bodyCamera.frameData.bin',
 'raw_video_data/_iblrig_bodyCamera.raw.mp4',
 'raw_video_data/_iblrig_leftCamera.frameData.bin',
 'raw_video_data/_iblrig_leftCamera.raw.mp4',
 'raw_video_data/_iblrig_rightCamera.frameData.bin',
 'raw_video_data/_iblrig_rightCamera.raw.mp4']

In [35]:
# Download raw video, return path
fpath_video = one.load_dataset(eid, f'*leftCamera.raw*', collection='raw_video_data')

100%|█████████████████████████████████████████| 15/15.0 [00:08<00:00,  1.76it/s]


In [36]:
fpath_video

PosixPath('/home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-06296/2023-07-13/001/raw_video_data/_iblrig_leftCamera.raw.mp4')

In [19]:
# Use io.video to stream video data instead of dodwnloading full videos
import ibllib.io.video as vidio
label = 'body' # 'left', 'right' or 'body'

# Find url of video data to stream
url = vidio.url_from_eid(eid, one=one)[label]
# Load video timestamps
ts = one.load_dataset(eid, f'*{label}Camera.times*', collection='alf')
# Find the frame closest to 1000s into data
frame_n = np.searchsorted(ts, 1000)
# Stream the data
frame = vidio.get_video_frame(url, frame_n)
print(frame.shape)

# Load the first 10 video frames
frames = vidio.get_video_frames_preload(url, range(10))
print(frames.shape)

(512, 640, 3)
(10, 512, 640, 3)0


In [20]:
# Get video metadata
meta = vidio.get_video_meta(url, one=one)
for k, v in meta.items():
    print(f'The video {k} = {v}')

The video length = 1682400
The video fps = 30
The video width = 640
The video height = 512
The video duration = 15:34:40
The video size = 12179980136


In [ ]:
## TODO: None of this seems to work (can't find data files)
# Use io.camera module to run video QC
from ibllib.qc.camera import CameraQC
label = 'left'

# Run right camera QC, downloading all but video file
qc = CameraQC(eid, camera=label, one=one)
qc.load_data()
# Run specific video QC check and display the plots
qc.load_data(download_data=True)
qc.check_position(display=True)  # NB: Not all checks make plots
# qc.run()

from ibllib.qc.camera import run_all_qc
# Run the QC for all cameras
qcs = run_all_qc(eid)

2025-01-15 15:35:48 INFO     camera.py:399  Computing QC outcome for left camera, session d77dd225-f7a9-4d01-996b-82c3ae73d56c
2025-01-15 15:35:48 INFO     camera.py:203  Gathering data for QC
2025-01-15 15:35:48 WARNING  raw_data_loaders.py:97   No data loaded: could not find raw data file
2025-01-15 15:35:48 WARNING  raw_data_loaders.py:97   No data loaded: could not find raw data file


TypeError: 'NoneType' object is not iterable

> /home/crombie/miniconda3/envs/iblenv/lib/python3.10/site-packages/ibllib/io/raw_data_loaders.py(799)load_bpod_fronts()
    797     BNC1_fronts = np.array([[np.nan, np.nan]])
    798     BNC2_fronts = np.array([[np.nan, np.nan]])
--> 799     for tr in data:
    800         BNC1_fronts = np.append(
    801             BNC1_fronts,



ipdb>  c
